In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from string import ascii_lowercase

In [23]:
files = []
for s in ascii_lowercase[:6]:
    files.append("{}.txt".format(s))

In [30]:
# Read the input
def read_input(filename):
    with open(filename, 'r') as f:
        w, h = [int(x) for x in f.readline().split()]
        office = []
        for i in range(h):
            office.append(f.readline())
        d = int(f.readline())
        devs = []
        for i in range(d):
            line = f.readline()
            devs.append({
                "company": line.split()[0],
                "bonus": int(line.split()[1]),
                "skills": set([x for x in line.split()[3:]])
                })
        pm = int(f.readline())
        managers = []
        for i in range(pm):
            line = f.readline()
            managers.append({
                "company": line.split()[0],
                "bonus": int(line.split()[1])
                })
        return office, devs, managers

In [45]:
for filename in files:
    office, devs, managers = read_input(filename)
    df_devs = pd.DataFrame(devs)
    df_managers = pd.DataFrame(managers)
    print(filename)
    print("#Developers: {}".format(len(df_devs)))
    print("Max devs per company: {}".format(
        df_devs.groupby('company').count().max()['bonus']
    ))
    print("# companies: {}".format(len(df_devs['company'].unique())))
    print("avg bonus: {} +- {}".format(df_devs['bonus'].mean(), df_devs['bonus'].std()))
    print("avg skills: {} +- {}".format(df_devs['skills'].apply(lambda x: len(x)).mean(), 
                                       df_devs['skills'].apply(lambda x: len(x)).std()))
    print("#Managers: {}".format(len(df_managers)))
    print("# Companies: {}".format(len(df_managers['company'].unique())))
    print("avg bonus: {} +- {}".format(df_managers['bonus'].mean(), df_managers['bonus'].std()))

a.txt
#Developers: 10
Max devs per company: 3
# companies: 5
avg bonus: 4.2 +- 2.4404006956964173
avg skills: 2.6 +- 0.8432740427115678
#Managers: 3
# Companies: 3
avg bonus: 2.6666666666666665 +- 2.081665999466133
b.txt
#Developers: 6836
Max devs per company: 90
# companies: 100
avg bonus: 10.515359859566999 +- 2.2992440860400762
avg skills: 14.95655354008192 +- 3.155332370822301
#Managers: 1207
# Companies: 100
avg bonus: 17.897265948632974 +- 6.398023960180999
c.txt
#Developers: 7588
Max devs per company: 89
# companies: 120
avg bonus: 5.0 +- 0.0
avg skills: 9.628887717448603 +- 6.870154129920177
#Managers: 1242
# Companies: 120
avg bonus: 5.0 +- 0.0
d.txt
#Developers: 30114
Max devs per company: 108
# companies: 350
avg bonus: 10.444876137344757 +- 2.409639498790193
avg skills: 15.010792322507804 +- 3.165728063881842
#Managers: 5049
# Companies: 350
avg bonus: 17.491186373539314 +- 6.443553791063172
e.txt
#Developers: 38165
Max devs per company: 90
# companies: 600
avg bonus: 5.130